In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-01-05 01:00:00')


In [100]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.
2024-01-04 17:29:39,957 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.


/mnt/c/Users/nisar/taxi_demand_predictor/.venv/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning:

ssl.PROTOCOL_TLS is deprecated



Finished: Reading data from Hopsworks, using Hive (85.03s) 


In [101]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.5883696791684192, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5883696791684192
[LightGBM] [Warning] bagging_fraction is set=0.7783852005158483, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7783852005158483


In [102]:
predictions['pickup_hour'] = current_date
predictions['pickup_ts'] = predictions['pickup_hour'].astype(int) // 10**6
predictions

,pickup_location_id,predicted_demand,pickup_hour,pickup_ts
0,1,0.0,2024-01-05 01:00:00,1704416400000
1,2,0.0,2024-01-05 01:00:00,1704416400000
2,3,0.0,2024-01-05 01:00:00,1704416400000
3,4,4.0,2024-01-05 01:00:00,1704416400000
4,5,0.0,2024-01-05 01:00:00,1704416400000
...,...,...,...,...
257,261,3.0,2024-01-05 01:00:00,1704416400000
258,262,4.0,2024-01-05 01:00:00,1704416400000
259,263,9.0,2024-01-05 01:00:00,1704416400000
260,264,7.0,2024-01-05 01:00:00,1704416400000


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [103]:
from src.feature_store_api import get_or_create_feature_group
from src.config import FEATURE_GROUP_PREDICTIONS_METADATA, MODEL_NAME

# connect to the feature group
feature_group = get_or_create_feature_group(FEATURE_GROUP_PREDICTIONS_METADATA)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.


In [104]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/262 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/305300/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f1e0980aed0>, None)